In [127]:
import os
import glob
os.chdir('/home/andrew/ImageSegmentation/Training')

In [132]:
LIMIT_TO_APPROVED = True
EXPERIMENT = 'SegmentationOfCells'
TRIAL = 6
STORE_DIRECTORY = 'PNG_1024'

BUCKET = 'duennerstore'
OUTPUT = 'segmented_data'

All masks have the name SemImage###_XYZ*.png. We find the masks by recursively searchinf for files of this type. Limit search to folders starting with 'Trial'

In [133]:
file_list_full =[file for file in glob.glob('**/SemImage*_*.png', recursive=True)
        if os.path.dirname(file).startswith('Trial')]

if LIMIT_TO_APPROVED == True:
    approved_file_list = [file.split('/')[-1] for file in glob.glob('Approved/*.png')]
    file_list = [file for file in file_list_full
                 if file.split('/')[-1] in approved_file_list]
else:
    file_list = file_list_full

In [134]:
mask_names  = []
mask_paths  = []
mask_image_ids   = []
mask_ids    = []
trials = []

for file in file_list:
    file_name = file.split('/')[-1]
    mask_names.append(file_name)
    mask_id = file_name.split('_')[-1].strip('.png')
    mask_ids.append(mask_id)
    path = os.path.abspath(file)
    trial = int(path.split('/')[5].strip('Trial'))
    trials.append(trial)
    mask_paths.append(path)
    image_number = int(file_name.split('_')[0].strip('SemImage'))
    mask_image_ids.append(image_number)

    
import pandas as pd
data = {
    'Image_ID': mask_image_ids,
    'Mask_ID': mask_ids,
    'Mask': mask_names,
    'Mask_Path': mask_paths,
    'Trials': trials
}

maskdf = pd.DataFrame(data)
maskdf

,Image_ID,Mask_ID,Mask,Mask_Path,Trials
0,87,HZ1TW,SemImage087_HZ1TW.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
1,10,D5XS0,SemImage010_D5XS0.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
2,27,R8J9U,SemImage027_R8J9U.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
3,57,ZR3PM,SemImage057_ZR3PM.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
4,127,AM1OG,SemImage127_AM1OG.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
5,165,A5SBA,SemImage165_A5SBA.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
6,178,ACNLR,SemImage178_ACNLR.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
7,74,VE7O9,SemImage074_VE7O9.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
8,31,URXPF,SemImage031_URXPF.png,/home/andrew/ImageSegmentation/Training/Trial5...,5
9,91,Q59XS,SemImage091_Q59XS.png,/home/andrew/ImageSegmentation/Training/Trial5...,5


In [137]:

data = pd.read_csv("/home/andrew/ImageSegmentation/Training/Trial5/Batch_3599747_batch_results.csv")
data['ImageID']=data['Input.image_url'].apply(lambda s: s.split('/')[-1])
def mask_fun(in1, in2):
    return in1.split('/')[-1].strip('.png')+'_'+in2[-5:]+'.png'

data['Mask'] = data.apply(lambda row: mask_fun(row['Input.image_url'], row['AssignmentId']),axis=1)
data = data[['Mask','ImageID','WorkerId', 'AssignmentId']]
maskdf = maskdf.merge(data)
maskdf

,Image_ID,Mask_ID,Mask,Mask_Path,Trials,ImageID,WorkerId,AssignmentId
0,87,HZ1TW,SemImage087_HZ1TW.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage087.png,A3V6XE43H0BFC9,33LKR6A5KEKN75R3YCS8NMRWUHZ1TW
1,10,D5XS0,SemImage010_D5XS0.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage010.png,A1XGHGHD0LCPXJ,33IZTU6J811UI6ZHG58P8QZBXD5XS0
2,27,R8J9U,SemImage027_R8J9U.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage027.png,A234WZ4KX8IW07,30MVJZJNHMDBI7YTQVTX0MM6UR8J9U
3,165,A5SBA,SemImage165_A5SBA.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage165.png,A2F9FURICZCOY3,304SM51WA34N53WGB9G7JT8JEA5SBA
4,74,VE7O9,SemImage074_VE7O9.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage074.png,A2F9FURICZCOY3,3TOK3KHVJTIL5Z1XRBNIUMCMYVE7O9
5,204,OT565,SemImage204_OT565.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage204.png,A1RECYT97C7ZWK,35H6S234SA0GTPHA3MMRF2CZMOT565
6,187,JN85Y,SemImage187_JN85Y.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage187.png,A182YKPS46KE9F,3Q5ZZ9ZEVOF3X30YW1JPZG8QSJN85Y
7,79,SX3O3,SemImage079_SX3O3.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage079.png,A1PGTQNCH0AIXV,31JLPPHS2UU34G0APZKSH506TSX3O3
8,191,QZQ5J,SemImage191_QZQ5J.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage191.png,A2F9FURICZCOY3,3GNA64GUZE49370UIV0PL7TQMQZQ5J
9,44,KFLJH,SemImage044_KFLJH.png,/home/andrew/ImageSegmentation/Training/Trial5...,5,SemImage044.png,A182YKPS46KE9F,386PBUZZXFX6J9Z2FSWN5VD1UKFLJH


In [136]:
import numpy as np
cdir = os.getcwd()
os.chdir('/home/andrew/ImageSegmentation/Training/Raw_Images_1024')
image_names = glob.glob('SemImage*.png')
os.chdir(cdir)
image_ids = []

for idx,image in enumerate(image_names):
    image_id = int(image.strip('SemImage').strip('.png'))
    image_ids.append(image_id)

image_ids = np.array(image_ids,'int')
image_mask_count = np.zeros((image_ids.shape),'int')
mask_image_paths=[]
mask_array = maskdf.values

masked_images = np.array(image_ids,'int')

mask_names = []
mask_trials = []

for idx,img in enumerate(image_ids):
    mask_images = []
    mask_count=0
    mask_image_rows = np.where(img==mask_array[:,0])
    
    these_mask_names = []
    these_mask_paths = []
    these_mask_trials = []
    
    mask_image_paths.append([])
    mask_names.append([])
    mask_trials.append([])
    
    if len(mask_image_rows[0])>0:
        image_mask_count[idx] = len(mask_image_rows[0])
        for row in mask_image_rows[0]:
            these_mask_names.append(mask_array[row,2])
            these_mask_paths.append(mask_array[row,3])
            these_mask_trials.append(mask_array[row,4])
    else:
        these_mask_names.append('none')
        these_mask_paths.append('N/A')
        these_mask_paths.append('')
    mask_names[idx].append(these_mask_names)
    mask_image_paths[idx].append(these_mask_paths)
    mask_trials[idx].append(these_mask_trials)

image_data = {
    'Image_ID': image_ids,
    'Image_Name': image_names,
    'Num_Masks': image_mask_count,
    'Mask_Names': mask_names,
    'Mask_Paths': mask_image_paths,
    'Mask_Trials': [val for sublist in mask_trials for val in sublist]
}

image_DF = pd.DataFrame(image_data,index=image_ids)

# Images with at least one mask

,Image_ID,Image_Name,Num_Masks,Mask_Names,Mask_Paths,Mask_Trials
101,101,SemImage101.png,1,[[SemImage101_9GSBJ.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
10,10,SemImage010.png,1,[[SemImage010_D5XS0.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
105,105,SemImage105.png,1,[[SemImage105_LA58M.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
95,95,SemImage095.png,2,"[[SemImage095_HZD4K.png, SemImage095_TTUR5.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[5, 5]"
35,35,SemImage035.png,1,[[SemImage035_YZTPO.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
31,31,SemImage031.png,2,"[[SemImage031_URXPF.png, SemImage031_CCEKK.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[5, 5]"
70,70,SemImage070.png,1,[[SemImage070_JOCT5.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
109,109,SemImage109.png,1,[[SemImage109_XTP4V.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
18,18,SemImage018.png,1,[[SemImage018_R4Y08.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]
212,212,SemImage212.png,1,[[SemImage212_5A1KU.png]],[[/home/andrew/ImageSegmentation/Training/Tria...,[5]


# Images without Mask

In [24]:
img_no_mask = image_DF.loc[image_DF['Num_Masks'] == 0]
img_no_mask

,Image_ID,Image_Name,Num_Masks,Mask_Names,Mask_Paths,Mask_Trials
34,34,SemImage034.png,0,[[none]],"[[N/A, ]]",[]
235,235,SemImage235.png,0,[[none]],"[[N/A, ]]",[]
24,24,SemImage024.png,0,[[none]],"[[N/A, ]]",[]
81,81,SemImage081.png,0,[[none]],"[[N/A, ]]",[]
244,244,SemImage244.png,0,[[none]],"[[N/A, ]]",[]
233,233,SemImage233.png,0,[[none]],"[[N/A, ]]",[]
85,85,SemImage085.png,0,[[none]],"[[N/A, ]]",[]
28,28,SemImage028.png,0,[[none]],"[[N/A, ]]",[]
144,144,SemImage144.png,0,[[none]],"[[N/A, ]]",[]
93,93,SemImage093.png,0,[[none]],"[[N/A, ]]",[]


In [25]:
images_without_mask = np.sort(img_no_mask.values[:,0])

# Select 50 of the unsorted images

In [26]:
num_images_to_classify = len(images_without_mask)
batch_size = 50
image_selection_gap = num_images_to_classify//batch_size

next_image_list=[]
for img_id in range(50):
    next_image_list.append(images_without_mask[image_selection_gap*img_id])

In [27]:
next_image_list

[7,
 11,
 15,
 19,
 23,
 28,
 32,
 37,
 41,
 46,
 51,
 55,
 59,
 63,
 68,
 72,
 77,
 81,
 85,
 89,
 93,
 98,
 103,
 107,
 111,
 115,
 119,
 123,
 129,
 133,
 137,
 141,
 146,
 151,
 155,
 160,
 164,
 168,
 172,
 177,
 181,
 185,
 190,
 194,
 198,
 203,
 207,
 211,
 215,
 220]

# Generate manifest for next batch

In [28]:
import csv

csv_file = EXPERIMENT+str(TRIAL)+'.csv'

if os.path.exists(csv_file):
    os.remove(csv_file)

with open(csv_file, 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',',  quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['image_url'])
    for img_num in next_image_list:
        img_name = 'SemImage{0:03d}.png'.format(img_num)
        bucket_loc = 'https://s3.amazonaws.com/'+BUCKET+'/'+STORE_DIRECTORY+'/'+img_name
        print(bucket_loc)
        csv_writer.writerow([bucket_loc])


https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage007.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage011.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage015.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage019.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage023.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage028.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage032.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage037.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage041.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage046.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage051.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage055.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage059.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage063.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage068.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage

4

,ImageID,MaskID,WorkerId
0,SemImage022.png,SemImage0225601L.png,A24DTU9HOGIJC1
1,SemImage022.png,SemImage022D31XV.png,A1LD9ZQ3XTCWUN
2,SemImage031.png,SemImage031CCEKK.png,A1RECYT97C7ZWK
3,SemImage031.png,SemImage031M9W9S.png,A24DTU9HOGIJC1
4,SemImage035.png,SemImage035OMXMN.png,A2F9FURICZCOY3
5,SemImage035.png,SemImage035BB0LR.png,A24DTU9HOGIJC1
6,SemImage040.png,SemImage040G10MT.png,AANUUKQPVMD9Q
7,SemImage040.png,SemImage040TE4Q0.png,A24DTU9HOGIJC1
8,SemImage044.png,SemImage044KFLJH.png,A182YKPS46KE9F
9,SemImage048.png,SemImage048M70PM.png,A3DF70OJSXWOJ2
